In [3]:
""" A La Carte GP and basis function demo. """

import matplotlib.pyplot as pl

import sys
sys.path.append("/root/dora-0.1-py3.4.egg")
import dora.regressors.gp as gp
import numpy as np
import logging

from revrand import basis_functions, regression, glm, likelihoods
from revrand.validation import mll, smse
from revrand.utils.datasets import gen_gausprocess_se

# Set up logging
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

In [5]:
# Algorithmic properties
nbases = 100
lenscale = 1  # For all basis functions that take lengthscales
lenscale2 = 0.5  # For the Combo basis
noise = 1
order = 5  # For polynomial basis
rate = 0.9
eta = 1e-5
passes = 500
batchsize = 100
reg = 1
usegradients = True
diagcov = False
rank = 0
N = 500
Ns = 250

In [7]:
# Dataset selection
# dataset = 'sinusoid'
dataset = 'gp1D'

# Dataset properties
lenscale_true = 0.7  # For the gpdraw dataset
noise_true = 0.1

basis = 'RKS'

In [8]:
# Sinusoid
if dataset == 'sinusoid':
    Xtrain = np.linspace(-2 * np.pi, 2 * np.pi, N)[:, np.newaxis]
    ytrain = np.sin(Xtrain).flatten() + np.random.randn(N) * noise
    Xtest = np.linspace(-2 * np.pi, 2 * np.pi, Ns)[:, np.newaxis]
    ftest = np.sin(Xtest).flatten()

# Random RBF GP
elif dataset == 'gp1D':

    Xtrain, ytrain, Xtest, ftest = \
        gen_gausprocess_se(N, Ns, lenscale=lenscale_true, noise=noise_true)

else:
    raise ValueError('Invalid dataset!')

Make Bases

In [9]:
if basis == 'FF':
    base = basis_functions.FastFood(nbases, Xtrain.shape[1])
elif basis == 'RKS':
    base = basis_functions.RandomRBF(nbases, Xtrain.shape[1])
elif basis == 'RBF':
    base = basis_functions.RadialBasis(Xtrain)
elif basis == 'Linear':
    base = basis_functions.LinearBasis(onescol=True)
elif basis == 'Poly':
    base = basis_functions.PolynomialBasis(order)
elif basis == 'Combo':
    base1 = basis_functions.RandomRBF(nbases, Xtrain.shape[1])
    base2 = basis_functions.LinearBasis(onescol=True)
    base3 = basis_functions.FastFood(nbases, Xtrain.shape[1])
    base = base1 + base2 + base3
else:
    raise ValueError('Invalid basis!')

Make real GP:

In [10]:
kdef = lambda h, k: h(1e-5, 1e2, 1) * k('gaussian', h(1e-5, 1e5, lenscale))
kfunc = gp.compose(kdef)

# Set up optimisation
learning_params = gp.OptConfig()
learning_params.sigma = gp.auto_range(kdef)
learning_params.noise = gp.Range([1e-5], [1e5], [1])
learning_params.walltime = 60


Learn regression parameters and predict

In [12]:
 if basis == 'Linear' or basis == 'Poly':
    hypers = ()
elif basis == 'FF' or basis == 'RKS' or basis == 'RBF':
    hypers = (lenscale,)
elif basis == 'Combo':
    hypers = (lenscale, lenscale2)
else:
    raise ValueError('Invalid basis!')

# combine training data to single array
train_data = np.hstack((ytrain[:, np.newaxis], Xtrain))

n_partitions = 4
data = sc.parallelize(train_data, n_partitions)
params_sgd = regression.learn_sgd_spark(data, base, hypers,
                                  var=noise**2, rate=rate, eta=eta,
                                  passes=passes, regulariser=reg,
                                  rank=rank, batchsize=batchsize)
Ey_s, Vf_s, Vy_s = regression.predict(Xtest, base, *params_sgd)
Sy_s = np.sqrt(Vy_s)

params_elbo = regression.learn(Xtrain, ytrain, base, hypers,
                               diagcov=diagcov, var=noise**2,
                               usegradients=usegradients, regulariser=reg)
Ey_e, Vf_e, Vy_e = regression.predict(Xtest, base, *params_elbo)
Sy_e = np.sqrt(Vy_e)

AttributeError: 'builtin_function_or_method' object has no attribute '__code__'